In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [3]:
def soup_to_dico(_soup):
    temps = _soup.find('div', {"class" : "time__details"}).find_all('div')
    temps_preparation = temps[1].get_text()
    temps_repos = temps[3].get_text()
    temps_cuisson = temps[5].get_text()

    image = _soup.find('img',{"class":"lazyload mrtn-print-only"}).get("data-src")

    nb_personne = _soup.find("div",{"class":'mrtn-recette_ingredients-counter'}).get("data-servingsnb")

    difficulte = _soup.find_all('div',{"class": 'recipe-primary__item'})
    result = []
    for div in difficulte:
        # Trouver la balise <span> et extraire son texte
        span = div.find('span')
        if span:
            result.append(span)
    temps_total = result[0].get_text()
    difficulte= result[1].get_text()
    cout = result[2].get_text()

    instructions = _soup.find_all("div",{"class":'recipe-step-list__container'})
    etapes = []
    for i in range(len(instructions)):
        for p in instructions[i].find_all("p"):
            etapes.append(p.get_text())
    
    liste_ingredients = []

    ingredients = _soup.find_all("span",{"class":"card-ingredient-title"})
    ingredients_id = _soup.find_all("div",{"class":"card-ingredient-checkbox"})

    for i in range(len(ingredients)):
        nom = ingredients[i].find("span",{"class":"ingredient-name"}).get("data-ingredientnameplural")
        quantite_unite = ingredients[i].find("span", {"class":"card-ingredient-quantity"})
        quantite = quantite_unite.get("data-ingredientquantity")
        unite = quantite_unite.find("span", {"class":"unit"}).get("data-unitsingular")
        id_ingredient= ingredients_id[i].find("input").get("id")
        

        liste_ingredients.append({
            "nom":nom,
            "quantite": quantite,
            "unite":unite,
            "id": id_ingredient[5:]
        })

    recette_ = {
        "temps_preparation" : temps_preparation,
        "tems_repos" : temps_repos,
        "temps_cuisson": temps_cuisson ,
        "image" : image ,
        "nb_personne": nb_personne,
        "temps_total": temps_total ,
        "difficulte":difficulte,
        "cout": cout,
        "etapes": etapes,
        "ingredients": liste_ingredients
    }

    return recette_
    

Récupère les titres et les liens vers les entrées et les plats des recettes Top de marmiton

In [4]:
urls = [
    'https://www.marmiton.org/recettes/top-internautes-plat-principal.aspx',
    'https://www.marmiton.org/recettes/top-internautes-entree.aspx',
    "https://www.marmiton.org/recettes/top-internautes-dessert.aspx"
    ]

titres = []
liens = []

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    titre = soup.find_all('h4', {"class" : "recipe-card__title"})
    lien=soup.find_all('a', {"class" : "recipe-card-link"})
    

    titres.extend(titre)
    liens.extend(lien)


recettes = [] #titres, liens
for titre, lien in zip(titres, liens):
    recettes.append({
        "titre": titre.get_text().strip(),
        "lien": lien["href"]
    })

for recette in recettes:
    lien = recette['lien']
    id = re.findall(r'(\d+)', lien)[0]
    response_recette = requests.get(lien)
    soup_recette = BeautifulSoup(response_recette.content, "html.parser")
    recette['id'] = id
    recette.update(soup_to_dico(soup_recette))



<>:21: SyntaxWarning: invalid escape sequence '\d'
<>:21: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_538/1732393173.py:21: SyntaxWarning: invalid escape sequence '\d'
  """


'\nrecettes = [] #titres, liens\nfor titre, lien in zip(titres, liens):\n    recettes.append({\n        "titre": titre.get_text().strip(),\n        "lien": lien["href"]\n    })\n\nfor recette in recettes:\n    lien = recette[\'lien\']\n    id = re.findall(r\'(\\d+)\', lien)[0]\n    response_recette = requests.get(lien)\n    soup_recette = BeautifulSoup(response_recette.content, "html.parser")\n    recette[\'id\'] = id\n    #recette.update(soup_to_dico(soup_recette))\n'

### Zone de test

In [3]:
response_recipe = requests.get("https://www.marmiton.org/recettes/recette_lasagnes-a-la-bolognaise_18215.aspx").text

In [5]:
soup = BeautifulSoup(response_recipe, "html.parser")

In [42]:
image_link = soup.find('img',{"class":"lazyload mrtn-print-only"}).get("data-src")

In [43]:
image_link

'https://assets.afcdn.com/recipe/20180209/77484_w300h200c1cx2464cy1632cxt0cyt0cxb4928cyb3264.webp'